# Najprostszy model

##Wstęp
Piosenki nie mają przypisanych gatunków, musimy temu zarządzić. Ponieważ wcześniej tych danych nie mieliśmy, nie możemy ocenić, jak dobrze dobrany jest gatunek piosenki.
Jakość podziału na grupy oceniamy na podstawie danych, które uzyskaliśmy: ile gatunków występuje w otrzymanym zbiore, jaka jest liczba piosenk poszcególnego gatunku. Liczba tych gatunków musi być zbliżona do liczby najczęściej występujących, "głównych" gatunków, również staramy się uzyskać bardziej zbalansowany podział na gatunki, staramy się unikać pojawiania się grup o pewnym gatunku, do którego należą 1-2 piosenki.


In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.cluster import KMeans

In [2]:
!pip install pandas==1.5.2
!pip install joblib==1.2.0
from joblib import load, dump

In [3]:
tracks = load("dane_do_modelu/tracks_df_preprocessed")

In [4]:
tracks.head(2)

,id,name,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artists_genre
0,48SFtLr5URCI97X2Ynfdnc,Par Avion (Live) ( 2014 - Remaster) - Live; 20...,0,291227,0,2yTUYhIf8fxptTIy3KLuJD,2014,0.603,0.517,6,-8.504,0.0235,0.695,0.000003,0.744,0.327,96.181,"[rock, classic uk pop, pop]"
1,1y0U0HAe5QfTRzOsz74bOt,My Foolish Heart,25,166080,0,338mC0yGyX0C9of8QMJ5hK,1950-01-01,0.313,0.116,0,-12.645,0.0319,0.953,0.331000,0.161,0.255,74.071,"[rock, classic uk pop, pop]"


In [5]:
# for clustering
tracks_only_features = tracks[['danceability','key','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo']].copy()

In [6]:
# for determining the genre that characterizes the group
tracks_only_genres = tracks[['artists_genre']]

In [7]:
tracks_only_features.head(2)

,danceability,key,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.603,6,0.517,-8.504,0.0235,0.695,0.000003,0.744,0.327,96.181
1,0.313,0,0.116,-12.645,0.0319,0.953,0.331000,0.161,0.255,74.071


## Clustering: pierwsza próba
Zgodnie z założeniami, korzystamy z metody K średnich.
Na razie zapoznajemy się z tym, jak ogólnie będzie wyglądało przypisanie gatunków dla piosenek.
Optymalna liczba grup oraz parametry modelu grupowania zostaną dobrane później, na razie sprawdzane jest, czy da się uzyskać jakiekolwiek wyniki

In [8]:
number_of_clusters = 1000

In [9]:
X = np.array(tracks_only_features).astype(float)
model = KMeans(n_clusters=number_of_clusters)
clusters_for_tracks = model.fit_predict(X)

C:\Users\leras\Documents\IUM\venv\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


##Ustalenie gatunku,który odpowiada grupie
Na podstawie gatunków, w których artyści tworzą, za pomocą heurystyk, jaki gatunek występuje w grupie najczęściej, określamy gatunek całej grupy.

In [10]:
artists_genres = tracks_only_genres.values.tolist()

In [11]:
genres_per_cluster = []
for i in range(number_of_clusters):
  genres_per_cluster.append([])

Dla każdego utworu dostaliśmy listę gatunków, w których tworzy jej autor

In [12]:
print(artists_genres[0][0])

['rock', 'classic uk pop', 'pop']


In [13]:
for i in range(len(clusters_for_tracks)):
  genres = artists_genres[i][0]
  for genre in genres:
    # if (genre != 'pop'):
    #   genres_per_cluster[clusters_for_tracks[i]].append(genre)
    genres_per_cluster[clusters_for_tracks[i]].append(genre)
  

Dla każdej grupy mamy przypisane wszystkie wystąpienia gatunkow u autorów piosenek z tej grupy

In [14]:
print(genres_per_cluster[0])

['classic bollywood', 'desi pop', 'traditional_folk', 'sufi', 'dutch trance', 'edm', 'pop', 'dance', 'trance', 'pop', 'latin', 'pop', 'latin', 'vintage tango', 'rock', 'metal', 'rock', 'acoustic', 'piano rock', 'pop', 'classic swedish pop', 'country', 'danspunk', 'latin', 'pop', 'latin', 'fado', 'jazz', 'samba', 'tropicalia', 'jazz', 'pop', 'turkish soundtrack', 'boogaloo', 'latin', 'salsa international', 'latin', 'jazz', 'nova mpb', 'samba de roda', 'tropicalia', 'indie extremena', 'latin', 'rock', 'flute rock', 'metal', 'progressive rock', 'finnish dance pop', 'metal', 'finnish pop punk', 'suomi rock', 'rock', 'early us punk', 'punk', 'skate punk', 'rock', 'early us punk', 'punk', 'skate punk', 'blues', 'r&b', 'classic bollywood', 'traditional_folk', 'ghazal', 'canadian pop', 'pop', 'new_age', 'pop', 'traditional_folk', 'pop', 'k-pop boy group', 'pop', 'rock', 'ska', 'ska revival', 'baroque pop', 'easy_listening', 'rock', 'acoustic', 'sunshine pop', 'rock', 'country', 'acoustic', 'cl

In [15]:
def most_frequent(List):
  return max(set(List), key = List.count)

In [16]:
clusters_genres = []
for cluster in genres_per_cluster:
  clusters_genres.append(most_frequent(cluster))

Gatunek, występujący najczęściej w grupie określa gatunek grupy

In [17]:
print(len(set(clusters_genres)))
print(set(clusters_genres))

12
{'jazz', 'deep progressive trance', 'new_age', 'latin', 'rock', 'dance', 'pop', 'acoustic', 'metal', 'r&b', 'traditional_folk', 'easy_listening'}


## Wyniki
Nie najlepsze: bardzo niezbalansowane

In [18]:
tracks_genres=[]
for cluster in clusters_for_tracks:
  tracks_genres.append(clusters_genres[cluster])
tracks_with_genres = tracks.assign(genre=tracks_genres)

In [19]:
tracks_with_genres.head(5)

,id,name,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artists_genre,genre
0,48SFtLr5URCI97X2Ynfdnc,Par Avion (Live) ( 2014 - Remaster) - Live; 20...,0,291227,0,2yTUYhIf8fxptTIy3KLuJD,2014,0.603,0.517,6,-8.504,0.0235,0.6950,0.000003,0.7440,0.327,96.181,"[rock, classic uk pop, pop]",pop
1,1y0U0HAe5QfTRzOsz74bOt,My Foolish Heart,25,166080,0,338mC0yGyX0C9of8QMJ5hK,1950-01-01,0.313,0.116,0,-12.645,0.0319,0.9530,0.331000,0.1610,0.255,74.071,"[rock, classic uk pop, pop]",pop
2,7ij5kN8jwXr8fZD54M0xb6,Aleni Aleni,51,235974,0,48CUA59SDed3IdCctKndud,2015,0.684,0.839,4,-6.457,0.0658,0.1200,0.000000,0.3540,0.580,128.051,"[rock, classic uk pop, pop]",pop
3,6tM2QQSXvoOAEfH0cARToU,A Contraluz,28,230560,0,2TkVZG5xW47ZC0227ftg3q,2009-10-28,0.439,0.709,2,-6.803,0.0339,0.0584,0.000000,0.0803,0.495,155.018,"[rock, classic uk pop, pop]",pop
4,6UIcN1tiiGdd7oMMzNvyaP,Time,53,297502,0,7lbSsjYACZHn1MSDXPxNF2,2020-05-08,0.479,0.478,5,-8.884,0.1100,0.0601,0.000007,0.0813,0.117,100.040,"[rock, classic uk pop, pop]",pop


In [20]:
tracks_with_genres.groupby('genre').count()

,id,name,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artists_genre
genre,,,,,,,,,,,,,,,,,,
acoustic,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56
dance,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
deep progressive trance,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
easy_listening,543,543,543,543,543,543,543,543,543,543,543,543,543,543,543,543,543,543
jazz,175,175,175,175,175,175,175,175,175,175,175,175,175,175,175,175,175,175
latin,3128,3128,3128,3128,3128,3128,3128,3128,3128,3128,3128,3128,3128,3128,3128,3128,3128,3128
metal,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70
new_age,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19
pop,69457,69457,69457,69457,69457,69457,69457,69457,69457,69457,69457,69457,69457,69457,69457,69457,69457,69457


##Czy da sie to klasyfikować?


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [22]:
#print(tracks_only_features.values)
print(tracks_genres)

['pop', 'pop', 'pop', 'pop', 'pop', 'rock', 'pop', 'pop', 'pop', 'pop', 'pop', 'rock', 'pop', 'pop', 'pop', 'rock', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'rock', 'latin', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'latin', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'latin', 'easy_listening', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'rock', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'rock', 'rock', 'pop', 'pop', 'pop', 'pop', 'rock', 'pop', 'pop', 'pop', 'rock', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'latin', 'latin', 'pop', 'pop', 'rock', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'rock', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'rock', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'latin'

In [23]:
X = tracks_only_features.values
y = tracks_genres
# Split dataset into random train and test subsets:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

# Standardize features by removing mean and scaling to unit variance:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)



In [24]:
# Use the KNN classifier to fit data:
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)



KNeighborsClassifier()

In [25]:
# Predict y data with classifier: 
y_predict = classifier.predict(X_test)

# Print results: 
print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict))
classifier.score(X_test, y_test)

[[    6     0     0     0     1     0     0     0     3     0     0     0]
 [    0     0     0     0     0     0     0     0     0     1     0     0]
 [    0     0     0     0     0     1     0     0     0     0     0     0]
 [    0     0     0     8     0     1     0     0    95     0     2     0]
 [    2     0     0     0     6     1     0     0    27     0     1     0]
 [    1     0     0     2     2    50     0     0   552     0    12     1]
 [    0     0     0     0     0     0     0     0    12     0     3     0]
 [    0     0     0     0     0     2     0     4     0     0     0     0]
 [    1     0     0    16     2   140     2     0 13490     0   258     2]
 [    0     0     0     0     0     0     0     0    14     0     0     0]
 [    0     0     0     0     2    28     0     0  1522     0   154     1]
 [    0     0     0     0     0     3     0     0    37     0     5     6]]


C:\Users\leras\Documents\IUM\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\leras\Documents\IUM\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\leras\Documents\IUM\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                         precision    recall  f1-score   support

               acoustic       0.60      0.60      0.60        10
                  dance       0.00      0.00      0.00         1
deep progressive trance       0.00      0.00      0.00         1
         easy_listening       0.31      0.08      0.12       106
                   jazz       0.46      0.16      0.24        37
                  latin       0.22      0.08      0.12       620
                  metal       0.00      0.00      0.00        15
                new_age       1.00      0.67      0.80         6
                    pop       0.86      0.97      0.91     13911
                    r&b       0.00      0.00      0.00        14
                   rock       0.35      0.09      0.14      1707
       traditional_folk       0.60      0.12      0.20        51

               accuracy                           0.83     16479
              macro avg       0.37      0.23      0.26     16479
           weighted avg

0.8328175253352752

In [27]:
#save model
dump(classifier, 'modele/m1_classifier')

['modele/m1_classifier']